In [33]:
import json
import dill as pkl
import networkx as nx
import matplotlib.pyplot as plt

from arango import ArangoClient

In [26]:
# Establish Client Connection
client = ArangoClient(hosts='http://127.0.0.1:8529')

db = client.db('reddit_comp_db', username='root', password='Mango0521!')

In [41]:
# create the graph
if db.has_graph('reddit'):
    reddit = db.graph('reddit')
else:
    reddit = db.create_graph('reddit')

In [42]:
# Create the Posts collection
if reddit.has_vertex_collection('posts'):
    posts = reddit.vertex_collection('posts')
else:
    posts = reddit.create_vertex_collection('posts')

In [43]:
# Read posts collection
with open('arango_formed_data/posts_collection.pkl','rb') as file:
    posts_collection = pkl.load(file)

In [ ]:
# Insert data from the posts collection
cursor = db.aql.execute('FOR doc IN posts RETURN doc')

if not list(cursor):
    posts.import_bulk(posts_collection)
else:
    print('document already populated')

In [45]:
# Create the Comments collection
if reddit.has_vertex_collection('comments'):
    comments = reddit.vertex_collection('comments')
else:
    comments = reddit.create_vertex_collection('comments')

In [49]:
# Read comments collection
with open('arango_formed_data/comments_collection.pkl','rb') as file:
    comments_collection = pkl.load(file)

In [50]:
cursor = db.aql.execute('FOR doc IN comments RETURN doc')

if not list(cursor):
    comments.import_bulk(comments_collection)
else:
    print('document already populated')

In [51]:
# Create Commented on (post and comments) edge collection
if reddit.has_edge_definition('commented_on'):
    commented_on = reddit.edge_collection('commented_on')
else:
    commented_on = reddit.create_edge_definition(
        edge_collection='commented_on',
        from_vertex_collections=['comments'],
        to_vertex_collections=['posts']
    )


In [54]:
# Read commented on edge collection
with open('arango_formed_data/commented_on_edge_collection.pkl','rb') as file:
    commented_on_edge_collection = pkl.load(file)

In [55]:
cursor = db.aql.execute('FOR doc IN commented_on RETURN doc')

if not list(cursor):
    commented_on.import_bulk(commented_on_edge_collection)
else:
    print('document already populated')

In [56]:
# Create the users collection
if reddit.has_vertex_collection('users'):
    users = reddit.vertex_collection('users')
else:
    users = reddit.create_vertex_collection('users')

In [57]:
# Read users collection
with open('arango_formed_data/users_collection.pkl','rb') as file:
    users_collection = pkl.load(file)

In [58]:
cursor = db.aql.execute('FOR doc IN users RETURN doc')

if not list(cursor):
    users.import_bulk(users_collection)
else:
    print('document already populated')

In [59]:
# Create Posted (posts and users) edge collection
if reddit.has_edge_definition('posted'):
    posted = reddit.edge_collection('posted')
else:
    posted = reddit.create_edge_definition(
        edge_collection='posted',
        from_vertex_collections=['users'],
        to_vertex_collections=['posts']
    )

In [60]:
# Read posted edge collection
with open('arango_formed_data/posted_edge_collection.pkl','rb') as file:
    posted_edge_collection = pkl.load(file)

In [61]:
cursor = db.aql.execute('FOR doc IN posted RETURN doc')

if not list(cursor):
    posted.import_bulk(posted_edge_collection)
else:
    print('document already populated')

In [62]:
# Create Commented (comments and users) edge collection
if reddit.has_edge_definition('commented'):
    commented = reddit.edge_collection('commented')
else:
    commented = reddit.create_edge_definition(
        edge_collection='commented',
        from_vertex_collections=['users'],
        to_vertex_collections=['comments']
    )

In [63]:
# Read commented edge collection
with open('arango_formed_data/commented_edge_collection.pkl','rb') as file:
    commented_edge_collection = pkl.load(file)

In [64]:
cursor = db.aql.execute('FOR doc IN commented RETURN doc')

if not list(cursor):
    commented.import_bulk(commented_edge_collection)
else:
    print('document already populated')